In [1]:
import matplotlib.pylab as plt
import pandas as pd
import sqlite3
import json
from functools import partial
try:
    import baidu_api_ak
except ImportError:
    baidu_api_ak = None

In [2]:
db = "./data/haizhu@gz.houses.db.20170106"
con = sqlite3.connect(db)

query = """
SELECT 
"链家编号", "标题", "总价", "建筑面积", "经度", "纬度"
FROM houses;
"""

df = pd.read_sql(query, con)
df["单价"] = df["总价"] / df["建筑面积"]
fn = lambda x, pnt_pos: int(x*pnt_pos+0.5)/pnt_pos
map_fn = partial(fn, pnt_pos=10000.0)
df["经度_集中"] = df["经度"].map(map_fn)
df["纬度_集中"] = df["纬度"].map(map_fn)

In [3]:
api_pattern_file = "./baidu_api_pattern.html"
api_file = "./data/haizhu@gz.houses.html"

df_group = df.groupby([df["经度_集中"], df["纬度_集中"]]).mean()
df_group["单价"] = df_group["单价"].map(partial(fn, pnt_pos=10.0)) * 20.0
to_dict = lambda row : {"lng": row["经度_集中"], "lat": row["纬度_集中"], "count": row["单价"]}
data = [to_dict(df_group.ix[index]) for index in df_group.index]
with open(api_pattern_file) as pattern_f, open(api_file, "w") as api_f:
    pattern = pattern_f.read()
    data_repr = json.dumps(data, ensure_ascii=False)
    data_repr = data_repr.replace("},", "},\n")
    
    center_lnt = (df["经度"].max() + df["经度"].min())/2.0
    center_lat = (df["纬度"].max() + df["纬度"].min())/2.0
    
    ak = "您的密钥"
    if baidu_api_ak:
        ak = baidu_api_ak.ak
    query_str = "/api?v2.0&ak=%s" % ak
    pattern = pattern.replace("{{query_url}}", query_str)
    pattern = pattern.replace("{{center_lnt}}", str(center_lnt))
    pattern = pattern.replace("{{center_lat}}", str(center_lat))
    pattern = pattern.replace("{{points}}", data_repr)
    api_f.write(pattern)